In [1]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
# Load Data 
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True) # human readable feature values

#TODO this needs to be chnaged --> Ordinal Encoding !
categorial = ['Country','Workclass','Marital Status', 'Occupation','Relationship', 'Race', 'Sex']
for a in categorial:
    enc= OrdinalEncoder()
    enc.fit(np.array(X_display[a].values).reshape(-1, 1))
    temp= enc.transform(np.array(X_display[a].values).reshape(-1, 1))
   
    X_display[a]= temp.reshape(-1)

In [3]:
from carla.data.catalog.online_catalog import OnlineCatalog
dataset = OnlineCatalog('adult')

In [4]:
dataset

In [5]:

from carla.models.catalog import MLModelCatalog


training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)
data_name = "adult"
dataset = OnlineCatalog(data_name)

balance on test set 0.23883245958934032, balance on test set 0.2408256880733945
Epoch 0/9
----------
train Loss: 0.3642 Acc: 0.8290

test Loss: 0.3414 Acc: 0.8412

Epoch 1/9
----------
train Loss: 0.3376 Acc: 0.8417

test Loss: 0.3510 Acc: 0.8366

Epoch 2/9
----------
train Loss: 0.3322 Acc: 0.8426

test Loss: 0.3310 Acc: 0.8432

Epoch 3/9
----------
train Loss: 0.3306 Acc: 0.8444

test Loss: 0.3269 Acc: 0.8488

Epoch 4/9
----------
train Loss: 0.3291 Acc: 0.8465

test Loss: 0.3245 Acc: 0.8491

Epoch 5/9
----------
train Loss: 0.3278 Acc: 0.8449

test Loss: 0.3304 Acc: 0.8381

Epoch 6/9
----------
train Loss: 0.3265 Acc: 0.8472

test Loss: 0.3380 Acc: 0.8381

Epoch 7/9
----------
train Loss: 0.3266 Acc: 0.8478

test Loss: 0.3232 Acc: 0.8473

Epoch 8/9
----------
train Loss: 0.3258 Acc: 0.8459

test Loss: 0.3312 Acc: 0.8468

Epoch 9/9
----------
train Loss: 0.3259 Acc: 0.8479

test Loss: 0.3366 Acc: 0.8408



In [6]:
# load catalog model
model_type = "ann"
ml_model = MLModelCatalog(
    dataset,
    model_type=model_type,
    load_online=True,
    backend="pytorch"
)


In [7]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:25]

display(test_factual)

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.02174,0.0,...,0.0,1.0,1.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
6,0.438356,0.100061,0.266667,0.00000,0.0,...,1.0,0.0,1.0,0.0,1.0
7,0.479452,0.133519,0.533333,0.00000,0.0,...,0.0,1.0,0.0,1.0,0.0


In [8]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.380148,0.122800,0.878825,0.100582,0.078802,...,0.0,1.0,1.0,1.0,1.0
2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
7,0.499396,0.153422,0.553287,0.019956,0.019952,...,0.0,0.0,1.0,1.0,1.0


In [9]:
def semantic(counterfactual, classification, causal_model):
    '''Compare Labels'''
    
    pass

''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)
